<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
#!pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [2]:
import random
import numpy as np
import torch

seed = 2022

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [3]:
import pandas as pd

datadir = '../../../Data'

df_args = pd.read_csv(datadir + '/arguments-training.tsv', sep = '\t')
df_lbls = pd.read_table(datadir + '/labels-training.tsv')
df_train = df_args.merge(df_lbls, how="left", on="Argument ID")

df_args = pd.read_csv(datadir + '/arguments-validation.tsv', sep = '\t')
df_lbls = pd.read_table(datadir + '/labels-validation.tsv')
df_validation = df_args.merge(df_lbls, how="left", on="Argument ID")

#df_args = pd.read_csv(datadir + '/arguments-test.tsv', sep = '\t')
#df_lbls = pd.read_table(datadir + '/labels-validation.tsv')
#df_test = df_args.merge(df_lbls, how="left", on="Argument ID")
# print(df)

from datasets import Dataset, DatasetDict
train_dataset      = Dataset.from_pandas(df_train,      split="train")
validation_dataset = Dataset.from_pandas(df_validation, split="validation")
test_dataset       = Dataset.from_pandas(df_validation, split="test")
dataset            = DatasetDict({ "train": train_dataset, "validation": validation_dataset, "test": test_dataset })

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity', '__index_level_0__'],
        num_rows: 5393
    })
    validation: Dataset({
        features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism:

Let's check the first example of the training split:

In [5]:
example = dataset['train'][0]
example

{'Argument ID': 'A01002',
 'Conclusion': 'We should ban human cloning',
 'Stance': 'in favor of',
 'Premise': 'we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.',
 'Self-direction: thought': 0,
 'Self-direction: action': 0,
 'Stimulation': 0,
 'Hedonism': 0,
 'Achievement': 0,
 'Power: dominance': 0,
 'Power: resources': 0,
 'Face': 0,
 'Security: personal': 0,
 'Security: societal': 1,
 'Tradition': 0,
 'Conformity: rules': 0,
 'Conformity: interpersonal': 0,
 'Humility': 0,
 'Benevolence: caring': 0,
 'Benevolence: dependability': 0,
 'Universalism: concern': 0,
 'Universalism: nature': 0,
 'Universalism: tolerance': 0,
 'Universalism: objectivity': 0,
 '__index_level_0__': 0}

The dataset consists of arguments, labeled with one or more values. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [6]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Argument ID', 'Conclusion', 'Stance', 'Premise', '__index_level_0__']]
#labels.sort()
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Self-direction: thought',
 'Self-direction: action',
 'Stimulation',
 'Hedonism',
 'Achievement',
 'Power: dominance',
 'Power: resources',
 'Face',
 'Security: personal',
 'Security: societal',
 'Tradition',
 'Conformity: rules',
 'Conformity: interpersonal',
 'Humility',
 'Benevolence: caring',
 'Benevolence: dependability',
 'Universalism: concern',
 'Universalism: nature',
 'Universalism: tolerance',
 'Universalism: objectivity']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [7]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # take a batch of texts
    premise    = examples["Premise"]
    conclusion = examples["Conclusion"]
    # encode them
    encoding = tokenizer(conclusion, premise, padding="max_length", truncation=True, max_length=128)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(premise), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [8]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [10]:
tokenizer.decode(example['input_ids'])

'[CLS] we should ban human cloning [SEP] we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [11]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [12]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Security: societal']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [13]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [14]:
from transformers import AutoModelForSequenceClassification

def instantiate_model():
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                               problem_type="multi_label_classification",
                                                               output_hidden_states=False,
                                                               num_labels=len(labels),
                                                               id2label=id2label,
                                                               label2id=label2id)
    return model

model = instantiate_model()

#for name, param in model.named_parameters():
#    print(name, param.requires_grad)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [15]:
batch_size = 8
metric_name = "f1_m"
num_train_epochs = 15
freeze_layers_bert = False

In [16]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [17]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import multilabel_confusion_matrix
from transformers import EvalPrediction
import torch
import os
import csv

save_eval_result_df = None
def save_eval_results(predictions, labels=labels, evaluationResultsDir="evaluationResults", evaluationResultsFilename="run.tsv"):
    if (save_eval_result_df is None):
        return
    # make sure the shapes match...
    if len(save_eval_result_df.index) != len(predictions):
        raise Exception("Dataframe number of rows differs from number of predictions") 
    if not os.path.exists(evaluationResultsDir):
        os.makedirs(evaluationResultsDir)
    with open(os.path.join(evaluationResultsDir, evaluationResultsFilename), "w") as runFile:
        writer = csv.DictWriter(runFile, fieldnames = ["Argument ID"] + labels, delimiter = "\t")
        writer.writeheader()
        for index, row in save_eval_result_df.iterrows():
            r = { "Argument ID": row[0] }
            i = 0
            for lbl in labels:
                r[lbl] = int(predictions[index][i])
                i += 1
            writer.writerow(r)
    
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, true_labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # Save results...
    save_eval_results(y_pred)
    # finally, compute metrics
    y_true = true_labels
    precision_micro_average = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    recall_micro_average = recall_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    precision_macro_average = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    recall_macro_average = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    mcm = multilabel_confusion_matrix(y_true=y_true, y_pred=y_pred, labels=np.arange(len(labels)))
    # return as dictionary
    metrics = {'p': precision_micro_average,
               'r': recall_micro_average,
               'f1': f1_micro_average,
               'p_m': precision_macro_average,
               'r_m': recall_macro_average,
               'f1_m': f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'mcm': mcm.tolist()
              }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        true_labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [18]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [19]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2057,  2323,  7221,  2529, 18856, 13369,   102,  2057,  2323,
         7221,  2529, 18856, 13369,  2004,  2009,  2097,  2069,  3426,  4121,
         3314,  2043,  2017,  2031,  1037,  9129,  1997,  1996,  2168,  4286,
         2770,  2105,  2035,  3772,  1996,  2168,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [20]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6864, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0489, -0.4447,  0.5347,  0.1085, -0.1382,  0.0691, -0.0238, -0.4892,
         -0.4905,  0.0597, -0.2362, -0.8063,  0.0960,  0.1316,  0.1972,  0.0164,
         -0.1664,  0.2019,  0.9102, -0.3094]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [21]:
metrics = {
    'p': [],
    'r': [],
    'f1': [],
    'p_m': [],
    'r_m': [],
    'f1_m': [],
    'roc_auc': [],
    'accuracy': []
}

model = instantiate_model()

## Freeze layers...
if freeze_layers_bert:
    for name, param in model.named_parameters():
        if name.startswith(("bert.embeddings", "bert.encoder")):
            param.requires_grad = False
        print(name, param.requires_grad)

trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
)
trainer.train()
results = trainer.evaluate()
for key in metrics:
    metrics[key].append(results.get('eval_' + key))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,P,R,F1,P M,R M,F1 M,Roc Auc,Accuracy,Mcm
1,0.408000,0.353494,0.703076,0.247995,0.366659,0.381817,0.149501,0.178751,0.613446,0.046414,"[[[1620, 25], [204, 47]], [[1381, 19], [449, 47]], [[1758, 0], [138, 0]], [[1793, 0], [103, 0]], [[1147, 174], [248, 327]], [[1732, 0], [164, 0]], [[1764, 0], [127, 5]], [[1766, 0], [130, 0]], [[946, 191], [238, 521]], [[1254, 154], [191, 297]], [[1724, 0], [172, 0]], [[1436, 5], [447, 8]], [[1836, 0], [60, 0]], [[1769, 0], [127, 0]], [[1263, 0], [633, 0]], [[1628, 0], [268, 0]], [[1119, 90], [412, 275]], [[1761, 8], [78, 49]], [[1673, 0], [223, 0]], [[1525, 0], [370, 1]]]"
2,0.339400,0.335567,0.721086,0.338261,0.460501,0.477327,0.212483,0.263613,0.655950,0.085443,"[[[1636, 9], [209, 42]], [[1349, 51], [366, 130]], [[1758, 0], [138, 0]], [[1793, 0], [103, 0]], [[1247, 74], [295, 280]], [[1731, 1], [159, 5]], [[1744, 20], [105, 27]], [[1766, 0], [130, 0]], [[881, 256], [159, 600]], [[1316, 92], [218, 270]], [[1723, 1], [145, 27]], [[1413, 28], [388, 67]], [[1836, 0], [60, 0]], [[1769, 0], [127, 0]], [[1194, 69], [426, 207]], [[1628, 0], [268, 0]], [[1009, 200], [264, 423]], [[1752, 17], [67, 60]], [[1673, 0], [223, 0]], [[1511, 14], [358, 13]]]"
3,0.282400,0.330805,0.705831,0.378833,0.493041,0.541753,0.260898,0.322131,0.673511,0.093354,"[[[1610, 35], [167, 84]], [[1294, 106], [308, 188]], [[1751, 7], [127, 11]], [[1789, 4], [89, 14]], [[1254, 67], [325, 250]], [[1729, 3], [156, 8]], [[1745, 19], [100, 32]], [[1766, 0], [130, 0]], [[904, 233], [167, 592]], [[1301, 107], [204, 284]], [[1707, 17], [131, 41]], [[1358, 83], [311, 144]], [[1836, 0], [60, 0]], [[1768, 1], [127, 0]], [[1193, 70], [418, 215]], [[1628, 0], [268, 0]], [[1003, 206], [247, 440]], [[1750, 19], [55, 72]], [[1665, 8], [219, 4]], [[1506, 19], [341, 30]]]"
4,0.251000,0.338875,0.648193,0.423022,0.511942,0.550018,0.298097,0.351095,0.688381,0.076477,"[[[1578, 67], [153, 98]], [[1306, 94], [324, 172]], [[1751, 7], [126, 12]], [[1784, 9], [85, 18]], [[1151, 170], [225, 350]], [[1727, 5], [151, 13]], [[1723, 41], [84, 48]], [[1766, 0], [130, 0]], [[823, 314], [123, 636]], [[1235, 173], [176, 312]], [[1713, 11], [133, 39]], [[1339, 102], [296, 159]], [[1834, 2], [58, 2]], [[1768, 1], [127, 0]], [[1048, 215], [310, 323]], [[1616, 12], [262, 6]], [[1097, 112], [349, 338]], [[1751, 18], [61, 66]], [[1657, 16], [214, 9]], [[1434, 91], [282, 89]]]"
5,0.231500,0.345060,0.626431,0.456204,0.527934,0.514771,0.328047,0.371623,0.700695,0.079114,"[[[1575, 70], [145, 106]], [[1265, 135], [279, 217]], [[1742, 16], [117, 21]], [[1778, 15], [81, 22]], [[1142, 179], [217, 358]], [[1720, 12], [151, 13]], [[1685, 79], [66, 66]], [[1765, 1], [130, 0]], [[903, 234], [172, 587]], [[1258, 150], [187, 301]], [[1708, 16], [133, 39]], [[1303, 138], [270, 185]], [[1833, 3], [57, 3]], [[1767, 2], [127, 0]], [[1071, 192], [311, 322]], [[1606, 22], [259, 9]], [[881, 328], [209, 478]], [[1752, 17], [63, 64]], [[1644, 29], [211, 12]], [[1433, 92], [273, 98]]]"
6,0.188700,0.358475,0.611386,0.466111,0.528955,0.513145,0.339328,0.377140,0.703209,0.075422,"[[[1555, 90], [135, 116]], [[1289, 111], [298, 198]], [[1750, 8], [123, 15]], [[1782, 11], [87, 16]], [[1104, 217], [199, 376]], [[1701, 31], [142, 22]], [[1685, 79], [65, 67]], [[1766, 0], [130, 0]], [[918, 219], [194, 565]], [[1150, 258], [146, 342]], [[1702, 22], [129, 43]], [[1328, 113], [295, 160]], [[1834, 2], [58, 2]], [[1766, 3], [127, 0]], [[994, 269], [264, 369]], [[1582, 46], [245, 23]], [[1020, 189], [279, 408]], [[1752, 17], [61, 66]], [[1649, 24], [207, 16]], [[1350, 175], [211, 160]]]"
7,0.168500,0.366833,0.618174,0.461079,0.528193,0.553867,0.343725,0.391041,0.701849,0.072257,"[[[1581, 64], [143, 108]], [[1235, 165], [261, 235]], [[1747, 11], [118, 20]], [[1779, 14], [81, 22]], [[1182, 139], [253, 322]], [[1698, 34], [137, 27]], [[1684, 80], [59, 73]], [[1765, 1], [127, 3]], [[834, 303], [144, 615]], [[1195, 213], [160, 328]], [[1702, 22], [126, 46]], [[1321, 120], [295, 

***** Running Evaluation *****
  Num examples = 1896
  Batch size = 8
/usr/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-675
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-675/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-675/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-675/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-675/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1896
  Batch size = 8
/usr/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-def

***** Running Evaluation *****
  Num examples = 1896
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-9450
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-9450/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-9450/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-9450/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-9450/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1896
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-10125
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-10125/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-10125/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-10125/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-englis

## Results

In [22]:
import pandas as pd
import numpy as np

decimal_digits = 4

data = [[round(np.mean(metrics['p_m']), decimal_digits),
         round(np.mean(metrics['r_m']), decimal_digits),
         round(np.mean(metrics['f1_m']), decimal_digits),
         round(np.mean(metrics['roc_auc']), decimal_digits),
         round(np.mean(metrics['accuracy']), decimal_digits),
        ],
        [round(np.std(metrics['p_m']), decimal_digits),
         round(np.std(metrics['r_m']), decimal_digits),
         round(np.std(metrics['f1_m']), decimal_digits),
         round(np.std(metrics['roc_auc']), decimal_digits),
         round(np.std(metrics['accuracy']), decimal_digits),
        ]]

df = pd.DataFrame(data, index=["", "±"], columns=["P", "R", "F1", "Roc Auc", "Accuracy"])
df
# df.style.hide_index()

,P,R,F1,Roc Auc,Accuracy
,0.5161,0.355,0.3978,0.7051,0.0633
±,0.0000,0.000,0.0000,0.0000,0.0000


## Evaluate

After training, we evaluate our model on the validation set.

In [23]:
save_eval_result_df = df_validation

results = trainer.evaluate()
results

***** Running Evaluation *****
  Num examples = 1896
  Batch size = 8


{'eval_loss': 0.3956400752067566,
 'eval_p': 0.6039783001808319,
 'eval_r': 0.47271583582324267,
 'eval_f1': 0.5303458009880029,
 'eval_p_m': 0.5160524297401972,
 'eval_r_m': 0.3549570646060203,
 'eval_f1_m': 0.397817599480555,
 'eval_roc_auc': 0.7051326715632801,
 'eval_accuracy': 0.06329113924050633,
 'eval_mcm': [[[1556, 89], [134, 117]],
  [[1230, 170], [266, 230]],
  [[1748, 10], [120, 18]],
  [[1779, 14], [80, 23]],
  [[1135, 186], [216, 359]],
  [[1701, 31], [141, 23]],
  [[1680, 84], [63, 69]],
  [[1760, 6], [127, 3]],
  [[915, 222], [191, 568]],
  [[1250, 158], [192, 296]],
  [[1693, 31], [123, 49]],
  [[1293, 148], [265, 190]],
  [[1826, 10], [55, 5]],
  [[1762, 7], [127, 0]],
  [[955, 308], [243, 390]],
  [[1539, 89], [224, 44]],
  [[1014, 195], [298, 389]],
  [[1751, 18], [60, 67]],
  [[1624, 49], [199, 24]],
  [[1379, 146], [229, 142]]],
 'eval_runtime': 14.6236,
 'eval_samples_per_second': 129.654,
 'eval_steps_per_second': 16.207,
 'epoch': 15.0}

In [24]:
#!python evaluator.py --inputDataset evaluationLabels --inputRun evaluationResults --outputDataset .

## Save Results

In [25]:
save_eval_result_df = df_validation

## Inference

Let's test the model on a new sentence:

In [26]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [27]:
logits = outputs.logits
logits.shape

torch.Size([1, 20])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [28]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Achievement', 'Security: societal']


In [29]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
hidden_states = outputs.hidden_states
hidden_states